In [1]:
import json
import linecache
import time
import jsonlines
from datetime import datetime, date
import pickle
import os

In [2]:
starting_date = date(1990,1,1)
start_time = time.time()

arxiv_folder='arxiv-snapshot'
arxiv_json = os.path.join(arxiv_folder,"arxiv-metadata-oai-snapshot.json")

arxiv_optics_quantum_original=[]
arxiv_optics_quantum_modified=[]

with jsonlines.open(arxiv_json, 'r') as f:
    for id_of_abstract, line in enumerate(f):
        if line['categories'] in ['physics.optics','quant-ph']:
            arxiv_optics_quantum_original.append(line)  ## store the original one
        
            get_date = datetime.strptime(line['versions'][0]['created'], '%a, %d %b %Y %H:%M:%S %Z').date()
            paper_time = (get_date - starting_date).days
            arxiv_optics_quantum_modified.append([line['categories'],line['title'],line['abstract'],paper_time]) ## store modified one
        
elapsed_time = time.time() - start_time
print(f"sub arxiv: {id_of_abstract}")
print(f"Quantum and Optics: {len(arxiv_optics_quantum_original)}; Modified: {len(arxiv_optics_quantum_modified)}, Elapsed time: {elapsed_time} seconds")

sub arxiv: 2227429
Quantum and Optics: 78084; Modified: 78084, Elapsed time: 20.9117271900177 seconds


In [3]:
store_folder='domain_data'

with open(os.path.join(store_folder,'arxiv_optics_quantum_original.pkl'), 'wb') as f:
    pickle.dump(arxiv_optics_quantum_original, f)
    
with open(os.path.join(store_folder,'arxiv_optics_quantum_style_modified.pkl'), 'wb') as f:
    pickle.dump(arxiv_optics_quantum_modified, f)

In [4]:
with open(os.path.join(store_folder,'arxiv_optics_quantum_original.json'), 'w') as f:
    json.dump(arxiv_optics_quantum_original, f)
        
with open(os.path.join(store_folder,'arxiv_optics_quantum_style_modified.json'), 'w') as f:
    json.dump(arxiv_optics_quantum_modified, f)

## make only strings (title+abstract)

In [5]:
# ## (Read the modified metadata; [source, title, abstract, time])
### (Make each article in string, under certain replacements)

def get_single_article_string(article):
    
    curr_title=article[1] #'title'
    curr_abstract=article[2] #'abstract'
    
    replace_pairs=[['\n',' '],['-',' '],[' \" a','oa'],['\" a','ae'],['\"a','ae'],[' \" o','oe'],['\" o','oe'],['\"o','oe'],[' \" u','ue'],
                   ['\" u','ue'],['\"u','ue'],[' \' a','a'],[' \' e','e'],[' \' o','o'],["\' ", ""],["\'", ""],['  ',' '],['  ',' ']]
    
    article_string=(curr_title +' '+ curr_abstract).lower()
    
    for rep_pair in replace_pairs:
        #print(rep_pair)
        
        article_string=article_string.replace(rep_pair[0],rep_pair[1])
        #print(article_string)
        #print('\n')
    
    return article_string


def get_all_paper_strings(article_lists, folder_file):

    if os.path.exists(os.path.join(folder_file,'arxiv_optics_quantum_paper_strings.pkl')):
        with open(os.path.join(folder_file,'arxiv_optics_quantum_paper_strings.pkl'), "rb") as f:
            arxiv_optics_quantum_paper_strings = pickle.load(f)
            
    else:
        all_paper_strings=[]
        cc=0
        for id_of_paper in range(len(article_lists)):
            cc+=1
            #if (cc%3000)==0:
                #print(str(cc)+'/'+str(len(article_lists)))

            all_paper_strings.append(get_single_article_string(article_lists[id_of_paper]))

        with open(os.path.join(folder_file,'arxiv_optics_quantum_paper_strings.pkl'), "wb") as f:
            pickle.dump(all_paper_strings, f)
    
    return all_paper_strings    


all_article_strings=get_all_paper_strings(arxiv_optics_quantum_modified,folder_file="domain_data")